# Library

In [4]:
import pandas as pd
import numpy as np
import os
import re
import collections
import unidecode
#from gensim.parsing.preprocessing import remove_stopwords
import nltk
from nltk.corpus import stopwords
import itertools 
from nltk.tokenize import word_tokenize
from string import punctuation

from nltk import ngrams

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [62]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf

from tensorflow.keras import preprocessing
from tensorflow.keras import layers

tfds.disable_progress_bar()

In [65]:
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

import tensorflow_datasets as tfds
import tensorflow_text as tf_text

# Functions

In [6]:
def read_texts(path):
    data = []
    file_name = os.listdir(path)

    for name in file_name:
        if name.endswith('.txt'):
            with open(path + name,encoding="utf8") as f:
                text = f.read()
                data.append({'nombre':name.replace('.txt',''), 'texto':text})

    df = pd.DataFrame(data)
    return df

In [7]:
def clean_text(string):
    """
    A method to clean text 
    """
    
    # Removing the punctuations
    for x in string.lower(): 
        if x in punctuation:
            if x != '/':
                string = string.replace(x, "")
            else:
                string = string.replace(x, " ")
    
    string = unidecode.unidecode(string)

#     # Converting the text to lower
#     string = string.lower()

    # Removing stop words
    string = ' '.join([word for word in string.split() if word not in swords])

    # Cleaning the whitespaces
    string = re.sub(r'\s+', ' ', string).strip()

    return string 

In [8]:
#nltk.download('stopwords')
swords = list(set(stopwords.words('spanish')))

In [9]:
def sep_num_text(data):

    words = word_tokenize(data) 
    for w in words:
        if re.search(r'\d', w):
            ind = words.index(w)
            words[ind] = [''.join(g) for k, g in itertools.groupby(w, str.isalpha)]
    data = ' '.join([x if type(x) is not list else ' '.join(x) for x in words])
    
    return data

# Load Data

### Testing

220 clinical cases.

In [7]:
testing_abbr = pd.read_csv("../datasets/testing_set/clinical_cases.abbreviations.testing_set.tsv", sep = '\t')
testing_met = pd.read_csv("../datasets/testing_set/clinical_cases.metadata.testing_set.tsv", sep = '\t')
testing_rel = pd.read_csv("../datasets/testing_set/clinical_cases.relations.testing_set.tsv", sep = '\t')

In [4]:
testing_met.head()

,# Document_ID,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S1130-01082006000700014-1.txt,1.txt,1130-0108,2006-07-01,Revista Española de Enfermedades Digestivas v...,http://scielo.isciii.es/scielo.php?script=sci_...
1,S1130-01082007000300006-7.txt,7.txt,1130-0108,2007-03-01,Revista Española de Enfermedades Digestivas v...,http://scielo.isciii.es/scielo.php?script=sci_...
2,S1134-80462009000100005-1.txt,1.txt,1134-8046,2009-02-01,Revista de la Sociedad Española del Dolor v.1...,http://scielo.isciii.es/scielo.php?script=sci_...
3,S1137-66272014000300015-1.txt,1.txt,1137-6627,2014-12-01,Anales del Sistema Sanitario de Navarra v.37 ...,http://scielo.isciii.es/scielo.php?script=sci_...
4,S0365-66912004001200011-1.txt,1.txt,0365-6691,2004-12-01,Archivos de la Sociedad Española de Oftalmolog...,http://scielo.isciii.es/scielo.php?script=sci_...


In [5]:
testing_rel = testing_rel.reset_index()

In [6]:
testing_rel.columns = ['# Document_ID', 'Mention_A_type', 'Mention_A_StartOffset',
      'Mention_A', 'Relation_type', 'Mention_B_type',
       'Mention_B_StartOffset', 'Mention_B_EndOffset', 'Mention_B']

In [7]:
testing_rel = testing_rel.rename(columns = {'# Document_ID': 'doc_id'})

In [8]:
testing_rel.head()

,doc_id,Mention_A_type,Mention_A_StartOffset,Mention_A,Relation_type,Mention_B_type,Mention_B_StartOffset,Mention_B_EndOffset,Mention_B
0,S0211-69952013000500019-1,SHORT_FORM,3739,ARA II,SHORT-LONG,LONG_FORM,3695,3737,antagonista del receptor de angiotesina II
1,S0211-69952013000500019-1,SHORT_FORM,2793,PCR,SHORT-LONG,LONG_FORM,2798,2837,reacción en cadena de enzima polimerasa
2,S0365-66912004000600008-1,SHORT_FORM,406,AV,SHORT-LONG,LONG_FORM,390,404,agudeza visual
3,S0211-69952012000500025-1,SHORT_FORM,945,angio-TAC,SHORT-LONG,LONG_FORM,908,943,angiotomografía computarizada axial
4,S1130-05582017000200122-1,SHORT_FORM,940,RMN,SHORT-LONG,LONG_FORM,910,938,resonancia magnética nuclear


In [9]:
testing_rel.Relation_type.unique()

array(['SHORT-LONG', 'SHORT-NESTED', 'NESTED-LONG'], dtype=object)

In [10]:
testing_abbr = testing_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [11]:
testing_abbr.head()

,doc_id,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S0004-06142010000500014-1,2037,2044,16SrRNA,16s ribosomal rna,16s ribosomal rno
1,S0004-06142010000500014-1,1349,1351,M.,mycobacterium,mycobacterium
2,S0004-06142010000500014-1,1339,1342,PCR,polymerase chain reaction,polymerase chain reaction
3,S0004-06142010000500014-1,611,615,BHCG,beta-human chorionic gonadotropin,beta-humar chorionic gonadotropin
4,S0004-06142010000500014-1,594,597,CEA,carcinoembrionary antigen,carcinoembrionary antiger


In [12]:
testing_raw = read_texts("../datasets/testing_set/testing_set.raw_text/")

In [13]:
testing_raw.head()

,nombre,texto
0,S1139-76322009000700016-1,Paciente de sexo femenino de 13 años y 7 meses...
1,S0210-48062007000700015-1,Varón de 72 años con antecedentes personales d...
2,S0212-71992005000500009-1,"Varón de 81 años, con antecedentes de fibrilac..."
3,S0365-66912004001200011-1,Paciente varón de 52 años que acudió a urgenci...
4,S1130-01082009000900015-1,Varón de 54 años con episodios de pancreatitis...


### Backgound

2879 clinical cases. 220 will be use for clinical evaluation

In [14]:
back_met = pd.read_csv("../datasets/background_test/clinical_cases.metadata.background_set.tsv", sep = '\t')

In [15]:
back_met = back_met.rename(columns = {'# Document_ID': 'doc_id'})

In [16]:
back_met.head()

,doc_id,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S0325-00752010000100014.txt,1.txt,0325-0075,02/2010,Archivos argentinos de pediatría,http://www.scielo.org.ar/scielo.php?script=sci...
1,S0325-00752013000200014.txt,1.txt,0325-0075,04/2013,Archivos argentinos de pediatría,http://www.scielo.org.ar/scielo.php?script=sci...
2,S0325-00752011000400017.txt,1.txt,0325-0075,08/2011,Archivos argentinos de pediatría,http://www.scielo.org.ar/scielo.php?script=sci...
3,S0325-00752013000600022.txt,1.txt,0325-0075,12/2013,Archivos argentinos de pediatría,http://www.scielo.org.ar/scielo.php?script=sci...
4,S0325-00752008000500013.txt,1.txt,0325-0075,10/2008,Archivos argentinos de pediatría,http://www.scielo.org.ar/scielo.php?script=sci...


In [17]:
back_raw = read_texts("../datasets/background_test/background_test_set/")

In [18]:
back_raw.head()

,nombre,texto
0,S0212-71992004000800006-1,Mujer de 57 años de edad con antecedentes pers...
1,S0716-10182014000100007-1,Se recibió en el Hospital Veterinario Puente A...
2,S1130-01082009000200016-1,"Mujer de 47 años de edad, sin antecedentes de ..."
3,S0716-10182015000400016-1,"Escolar de 11 años de edad, género femenino, s..."
4,S1130-01082004001200010-2,"Varón de 25 años, remitido a nuestro Servicio ..."


### Development

146 clinical cases

In [19]:
dev_abbr = pd.read_csv("../datasets/development_set/clinical_cases.abbreviations.development_set.tsv", sep = '\t')
dev_met = pd.read_csv("../datasets/development_set/clinical_cases.metadata.development_set.tsv", sep = '\t')
dev_rel = pd.read_csv("../datasets/development_set/clinical_cases.relations.development_set.tsv", sep = '\t')

In [20]:
dev_met = dev_met.rename(columns = {'# Document_ID': 'doc_id'})

In [21]:
dev_met.head()

,doc_id,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S1130-14732005000300004-1.txt,1.txt,1130-1473,2005-06-01,Neurocirugía v.16 n.3 2005,http://scielo.isciii.es/scielo.php?script=sci_...
1,S1130-01082008000200009-1.txt,1.txt,1130-0108,2008-02-01,Revista Española de Enfermedades Digestivas v...,http://scielo.isciii.es/scielo.php?script=sci_...
2,S1137-66272012000300021-1.txt,1.txt,1137-6627,2012-12-01,Anales del Sistema Sanitario de Navarra v.35 ...,http://scielo.isciii.es/scielo.php?script=sci_...
3,S1699-695X2016000200010-1.txt,1.txt,1699-695X,2016-06-01,Revista Clínica de Medicina de Familia v.9 n....,http://scielo.isciii.es/scielo.php?script=sci_...
4,S1130-01082007001100009-1.txt,1.txt,1130-0108,2007-11-01,Revista Española de Enfermedades Digestivas v...,http://scielo.isciii.es/scielo.php?script=sci_...


In [22]:
dev_rel = dev_rel.reset_index()

In [23]:
dev_rel.columns = ['# Document_ID', 'Mention_A_type', 'Mention_A_StartOffset',
      'Mention_A', 'Relation_type', 'Mention_B_type',
       'Mention_B_StartOffset', 'Mention_B_EndOffset', 'Mention_B']

In [24]:
dev_rel = dev_rel.rename(columns = {'# Document_ID': 'doc_id'})

In [25]:
dev_rel.head()

,doc_id,Mention_A_type,Mention_A_StartOffset,Mention_A,Relation_type,Mention_B_type,Mention_B_StartOffset,Mention_B_EndOffset,Mention_B
0,S1888-75462015000400006-1,SHORT_FORM,1436,AP,SHORT-LONG,LONG_FORM,1419,1434,anteroposterior
1,S0210-56912009000700006-2,SHORT_FORM,876,angio-RMN,SHORT-LONG,LONG_FORM,858,874,angiorresonancia
2,S0210-56912009000700006-2,SHORT_FORM,819,RMN,SHORT-LONG,LONG_FORM,789,817,resonancia magnética nuclear
3,S1698-44472005000300015-1,SHORT_FORM,209,ATM,SHORT-LONG,LONG_FORM,177,207,articulación témporomandibular
4,S1698-44472005000300015-1,SHORT_FORM,1125,TC,SHORT-LONG,LONG_FORM,1099,1123,tomografía computerizada


In [26]:
dev_abbr = dev_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [27]:
dev_abbr.head()

,doc_id,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S1130-14732005000300004-1,1216,1218,C3,tercera vértebra cervical,tercero vértebra cervical
1,S1130-14732005000300004-1,717,719,C2,segunda vértebra cervical,segundo vértebra cervical
2,S1130-14732005000300004-1,3191,3193,C3,tercera vértebra cervical,tercero vértebra cervical
3,S1130-14732005000300004-1,2867,2869,C3,tercera vértebra cervical,tercero vértebra cervical
4,S1130-14732005000300004-1,2862,2864,C2,segunda vértebra cervical,segundo vértebra cervical


In [28]:
dev_raw = read_texts("../datasets/development_set/development_set.raw_text/")

In [29]:
dev_raw.head()

,nombre,texto
0,S1139-76322017000200010-1,Niña de dos años y diez meses con antecedentes...
1,S0365-66912005001100008-1,Se presenta el caso de un varón de 45 años que...
2,S1130-01082006001000017-1,Mujer de 42 años de edad con antecedentes pers...
3,S0212-71992005001200008-1,Paciente de 57 años con ingresos hospitalarios...
4,S0365-66912007000300010-1,Paciente de 33 años que el 20-08-05 es traslad...


### Sample test

15 clinical cases

In [30]:
sample_abbr = pd.read_csv("../datasets/sample_set/clinical_cases.abbreviations.sample_set.tsv", sep = '\t')
sample_met = pd.read_csv("../datasets/sample_set/clinical_cases.metadata.sample_set.tsv", sep = '\t')
sample_rel = pd.read_csv("../datasets/sample_set/clinical_cases.relations.sample_set.tsv", sep = '\t')

In [31]:
sample_met = sample_met.rename(columns = {'# Document_ID': 'doc_id'})

In [32]:
sample_met.head()

,doc_id,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S0004-06142006000900015-1,1,0004-0614,2006-11-01,Archivos Españoles de Urología (Ed. impresa) ...,http://scielo.isciii.es/scielo.php?script=sci_...
1,S0004-06142006000600015-1,1,0004-0614,2006-08-01,Archivos Españoles de Urología (Ed. impresa) ...,http://scielo.isciii.es/scielo.php?script=sci_...
2,S0004-06142007000700014-1,1,0004-0614,2007-09-01,Archivos Españoles de Urología (Ed. impresa) ...,http://scielo.isciii.es/scielo.php?script=sci_...
3,S0004-06142007000900013-1,1,0004-0614,2007-11-01,Archivos Españoles de Urología (Ed. impresa) ...,http://scielo.isciii.es/scielo.php?script=sci_...
4,S0004-06142006000200014-1,1,0004-0614,2006-03-01,Archivos Españoles de Urología (Ed. impresa) ...,http://scielo.isciii.es/scielo.php?script=sci_...


In [33]:
sample_rel = sample_rel.rename(columns = {'# Document_ID': 'doc_id'})

In [34]:
sample_rel.head()

,doc_id,Mention_A_type,Mention_A_StartOffset,Mention_A_EndOffset,Mention_A,Relation_type,Mention_B_type,Mention_B_StartOffset,Mention_B_EndOffset,Mention_B
0,S0004-06142006000700014-1,SHORT_FORM,926,929,CEA,SHORT-LONG,LONG_FORM,896,924,Antígeno Carcino Embrionario
1,S0004-06142005001000011-1,SHORT_FORM,1626,1629,ROT,SHORT-LONG,LONG_FORM,1600,1624,reflejos osteotendinosos
2,S0004-06142005001000011-1,SHORT_FORM,1715,1718,RMN,SHORT-LONG,LONG_FORM,1685,1713,resonancia magnética nuclear
3,S0004-06142005001000011-1,SHORT_FORM,1663,1666,RCP,SHORT-LONG,LONG_FORM,1639,1661,reflejo cutaneoplantar
4,S0004-06142005001000011-1,SHORT_FORM,1808,1811,LCR,SHORT-LONG,LONG_FORM,1783,1806,líquido cefalorraquídeo


In [35]:
sample_abbr = sample_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [36]:
sample_abbr.head()

,doc_id,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S0004-06142005001000011-1,1034,1036,Kg,kilogramo,kilogramo
1,S0004-06142005001000011-1,1031,1033,mg,miligramo,miligramo
2,S0004-06142005001000011-1,196,199,IgA,inmunoglobulina a,inmunoglobulina a
3,S0004-06142005001000011-1,2057,2060,LCR,líquido cefalorraquídeo,líquido cefalorraquídeo
4,S0004-06142005001000011-1,1594,1598,EEII,extremidades inferiores,extremidad inferior


In [37]:
sample_raw = read_texts("../datasets/sample_set/sample_set.raw_text/")

In [38]:
sample_raw.head()

,nombre,texto
0,S0004-06142006000600014-1,"Paciente varón, de 40 años de edad, con antece..."
1,S0004-06142006000300015-1,Paciente de 50 años con antecedente de litiasi...
2,S0004-06142007000900013-1,Presentamos el caso de un recién nacido de tre...
3,S0004-06142005001000011-1,Varón de 58 años de edad en el momento del tra...
4,S0004-06142006000700013-1,"Paciente varón, de 63 años de edad, mestizo, d..."


### Trainning

318 clinical cases

In [10]:
train_abbr = pd.read_csv("../datasets/trainning_set/clinical_cases.abbreviations.training_set.tsv", sep = '\t')

In [11]:
train_met = pd.read_csv("../datasets/trainning_set/clinical_cases.metadata.training_set.tsv", sep = '\t')
train_rel = pd.read_csv("../datasets/trainning_set/clinical_cases.relations.training_set.tsv", sep = '\t')

In [12]:
train_met = train_met.rename(columns = {'# Document_ID': 'doc_id'})

In [13]:
train_met.head()

,doc_id,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S1139-76322015000500009-1.txt,1.txt,1139-7632,2015-12-01,Pediatría Atención Primaria v.17 n.68 2015,http://scielo.isciii.es/scielo.php?script=sci_...
1,S1130-05582008000400007-2.txt,2.txt,1130-0558,2008-08-01,Revista Española de Cirugía Oral y Maxilofacia...,http://scielo.isciii.es/scielo.php?script=sci_...
2,S0210-48062006000100012-1.txt,1.txt,0210-4806,2006-01-01,Actas Urológicas Españolas v.30 n.1 2006,http://scielo.isciii.es/scielo.php?script=sci_...
3,S0213-12852003000500002-1.txt,1.txt,0213-1285,2003-10-01,Avances en Odontoestomatología v.19 n.5 2003,http://scielo.isciii.es/scielo.php?script=sci_...
4,S0212-71992005000400007-1.txt,1.txt,0212-7199,2005-04-01,Anales de Medicina Interna v.22 n.4 2005,http://scielo.isciii.es/scielo.php?script=sci_...


In [14]:
train_rel = train_rel.reset_index()

In [15]:
train_rel.columns = ['# Document_ID', 'Mention_A_type', 'Mention_A_StartOffset',
      'Mention_A', 'Relation_type', 'Mention_B_type',
       'Mention_B_StartOffset', 'Mention_B_EndOffset', 'Mention_B']

In [16]:
train_rel = train_rel.rename(columns = {'# Document_ID': 'doc_id'})

In [17]:
train_rel.head()

,doc_id,Mention_A_type,Mention_A_StartOffset,Mention_A,Relation_type,Mention_B_type,Mention_B_StartOffset,Mention_B_EndOffset,Mention_B
0,S1130-01082009000400014-1,SHORT_FORM,476,NPT,SHORT-LONG,LONG_FORM,454.0,474.0,nutrición parenteral
1,S1130-63432016000100009-1,SHORT_FORM,614,NIHSS,SHORT-LONG,LONG_FORM,621.0,662.0,National Institute of Health Stroke Scale
2,S1139-76322017000200007-1,SHORT_FORM,1145,CMV,SHORT-LONG,LONG_FORM,1128.0,1143.0,citomegalovirus
3,S1139-76322017000200007-1,SHORT_FORM,1243,VSG,SHORT-LONG,LONG_FORM,1206.0,1241.0,velocidad de sedimentación globular
4,S1139-76322017000200007-1,SHORT_FORM,1300,IGRA,SHORT-LONG,LONG_FORM,1267.0,1298.0,interferon-gamma release assays


In [18]:
train_abbr = train_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [19]:
train_abbr.Definition.nunique()

908

In [20]:
train_abbr.head()

,doc_id,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S0210-48062004000500008-1,1650,1652,ml,mililitro,mililitro
1,S0210-48062004000500008-1,708,709,l,litro,litro
2,S0210-48062004000500008-1,704,707,mEq,miliequivalente,miliequivalente
3,S0210-48062004000500008-1,677,681,pCO2,presión parcial de co2,presión parcial de co2
4,S0210-48062004000500008-1,2287,2290,HLA,human leucocyte antigen,human leucocyte antiger


In [21]:
train_raw = read_texts("../datasets/trainning_set/training_set.raw_text/")

In [22]:
train_raw = train_raw.rename(columns = {'nombre': 'doc_id'})

In [23]:
train_raw.head()

,doc_id,texto
0,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...
1,S0212-71992005000400009-1,"Se trataba de un varón de 27 años de edad, que..."
2,S0004-06142008000700015-2,Varón de 33 años fumador de un paquete de ciga...
3,S0210-56912006000800008-1,"Hombre de 42 años, bebedor de más de 100 g de ..."
4,S0376-78922009000300010-1,Paciente de 18 años de edad que 5 meses antes ...


## Opción 1

In [22]:
train_abbr.shape

(4260, 6)

In [13]:
train_raw.shape

(318, 2)

In [102]:
train = train_raw.merge(train_abbr[['doc_id', 'Abbreviation', 'Definition_lemmatized']], on = 'doc_id', how = 'left')

In [103]:
train = train.merge(train_abbr, on = ['doc_id', 'Abbreviation'], how = 'left')

In [104]:
train = train.drop_duplicates()

In [105]:
train = train[['doc_id', 'texto', 'Abbreviation', 'Definition_lemmatized_x', 'StartOffset', 'EndOffset']]

In [106]:
train = train.rename(columns = {'Definition_lemmatized_x':'Definition'})

In [107]:
train.head()

,doc_id,texto,Abbreviation,Definition,StartOffset,EndOffset
0,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...,PAAF,punción aspiración con aguja fino,1006.0,1010.0
1,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...,RM,resonancia magnético,789.0,791.0
2,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...,cm,centímetro,1345.0,1347.0
3,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...,cm,centímetro,1217.0,1219.0
4,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...,g,gramo,1304.0,1305.0


In [108]:
train.shape

(4379, 6)

In [109]:
train['doc_id'].nunique()

318

In [110]:
train['Abbreviation'].nunique()

768

In [23]:
# train['texto'] = train.apply(replace_sf_target, axis = 1)

In [111]:
train = train.rename(columns = {'Abbreviation': 'short_form', 'Definition': 'long_form'})

In [112]:
train = train.dropna(subset = ['short_form', 'long_form'])

In [113]:
train.shape

(4355, 6)

### Processing text

### 1. Tokenize and delete stop words

In [114]:
train['texto'] = train['texto'].map(clean_text)

In [115]:
train['long_form'] = train['long_form'].map(clean_text)

Separate words or short forms that contains numbers

In [116]:
train['texto'] = train['texto'].map(sep_num_text)

In [117]:
train = train.drop_duplicates()
train.shape

(4352, 6)

In [49]:
# train['text_tokenize'] = train['text_tokenize'].map(lambda x: word_tokenize(x))

In [144]:
train[['StartOffset', 'EndOffset']] = train[['StartOffset', 'EndOffset']].astype(int)

In [145]:
train.head()

,doc_id,texto,short_form,long_form,StartOffset,EndOffset
0,S1130-05582012000300005-1,Acude consultas paciente presenta tumoracion c...,PAAF,punción aspiración aguja fino,1006,1010
1,S1130-05582012000300005-1,Acude consultas paciente presenta tumoracion c...,RM,resonancia magnético,789,791
2,S1130-05582012000300005-1,Acude consultas paciente presenta tumoracion c...,cm,centímetro,1345,1347
3,S1130-05582012000300005-1,Acude consultas paciente presenta tumoracion c...,cm,centímetro,1217,1219
4,S1130-05582012000300005-1,Acude consultas paciente presenta tumoracion c...,g,gramo,1304,1305


Replace SF in the text for a label target

In [31]:
# def replace_sf_target(row):
#     row['texto'] = row['texto'].map(lambda x: x.replace(x, '<target>') if x == row['Abbreviation'] else x) 
#     return row['texto']

Transform dataframe to a list of dictionaries

In [146]:
train_data = train[['doc_id', 'texto', 'short_form', 'long_form', 'StartOffset', 'EndOffset']].to_dict('records')

For separate numbers from letters in the text without list phase

In [50]:
# def sep_num_let_intext(data):
    
#     for instance in data:
#         instance['texto'] = re.sub(r'(\d+)', r'\g<1> ', instance['texto'])
#     return data

### 2. Build vocabulary

In [119]:
def build_vocab(data):
    """
    return a dict with each word and its id. Id is assigned based on word frecuency, it means, it 0 will be assigned to a most fecuency word
    """
    
    #coge cada palabra de cada texto y la añade a una colección que tiene la palabra y su frecuencia de aparición
    counter = collections.Counter()
    for elem in data:
        counter.update(word_tokenize(elem['texto']))

    # remove infrequent words
    min_freq = 1
    filtered = [item for item in counter.items() if item[1]>=min_freq]

    count_pairs = sorted(filtered, key=lambda x: -x[1]) #ordena la lista filtered en orden descendente de frecuencia (que es el segundo elemento de la tupla)
    words, _ = list(zip(*count_pairs)) #separa las palabras de su frecuencia
    word_to_id = dict(zip(words, range(len(words)))) #crea un diccionario donde la clave es la palabra y el valor un índice. La palabra de índice 0 será la de mayor frecuencia de aparición

    return word_to_id

In [120]:
word_id_dict = build_vocab(train_data)

In [56]:
len(word_id_dict)

13090

### 3. Calculate sense Ids

In [121]:
def build_sense_ids(data):
    words = set()
    word_to_senses = {}
    for elem in data:
        target_word = elem['short_form']
        target_sense = elem['long_form']
        if target_word not in words:
            words.add(target_word) #añade la palabra al set de palabras
            word_to_senses.update({target_word: [target_sense]}) #añade la palabra al dict con su significado en una lista de valores
        else:
            if target_sense not in word_to_senses[target_word]:
                word_to_senses[target_word].append(target_sense) #si la palabra está en el dict pero no el significado, lo añade a la lista
    
    words = list(words)
    target_word_to_id = dict(zip(words, range(len(words)))) #diccionario de cada palabra ambigua con un un índice
    target_sense_to_id = [dict(zip(word_to_senses[word], range(len(word_to_senses[word])))) for word in words] #diccionario de cada significado y un índice

    n_senses_from_word_id = dict([(target_word_to_id[word], len(word_to_senses[word])) for word in words]) #diccionario índice de cada palabra ambigua y el número de significados diferentes
    return target_word_to_id, target_sense_to_id, len(words), n_senses_from_word_id

In [122]:
sf_to_id, lf_to_id, n_words, n_lf_from_sf_id = build_sense_ids(train_data)
print('Vocabulary size: %d' % len(word_id_dict))

Vocabulary size: 12942


### 4. Context for each target sense

In [123]:
def build_context(data, word_to_id):
    target_sense_to_context = {}
    for elem in data:
        target_sense_id = elem['long_form'] # 'id' or 'target_sense'?? DEBERIA SER TARGET SENSE ID
        context = word_tokenize(elem['texto'])
        #context = sparse_matrix(context, word_to_id)
        if target_sense_id not in target_sense_to_context:
            #target_sense_to_context.update({target_sense:context})
            target_sense_to_context[target_sense_id] = []
        target_sense_to_context[target_sense_id].append(context) #dict por sense id: contexto
    
    return target_sense_to_context

In [124]:
#build context vocab of the target sense
train_target_sense_to_context = build_context(train_data , word_id_dict)

### 5. Convert to numeric

Get forward and backward context of the ambiguous word

In [136]:
def get_f_b_context(data):
    
    '''Return context in a list of words'''
    
    all_data = []
    sf_not_found = []
    
    for instance in data:
        
        words = word_tokenize(instance['texto'])            
        target_word = instance['short_form'] 
        
        if target_word in words:

            stop_idx = words.index(target_word) #índice de la palabra target

            _instance = []
            xf = np.array(words[:stop_idx]) #palabras anteriores a la target
            xb = np.array(words[stop_idx+1:])    #palabras posteriores a la target            

            instance_id = instance['long_form'] #id del significado

            _instance.append(target_word)
            _instance.append(xf)
            _instance.append(xb)
            _instance.append(instance_id)

            all_data.append(_instance[:])
        else:
            sf_not_found.append(target_word)
#             print("El acrónimo {} no aparece en el texto {}".format(target_word, instance['doc_id']))
        
    return all_data, sf_not_found

## Opción 2: Buscar el contexto anterior y posterior con los Start y End Offsets

### Preprocesamiento dataset -> diccionario

In [24]:
train = train_raw.merge(train_abbr[['doc_id', 'Abbreviation', 'Definition_lemmatized']], on = 'doc_id', how = 'left')

In [25]:
train = train.merge(train_abbr, on = ['doc_id', 'Abbreviation'], how = 'left')

In [26]:
train = train.drop_duplicates()

In [27]:
train = train[['doc_id', 'texto', 'Abbreviation', 'Definition_lemmatized_x', 'StartOffset', 'EndOffset']]

In [28]:
train = train.rename(columns = {'Definition_lemmatized_x':'Definition'})

In [29]:
train.head()

,doc_id,texto,Abbreviation,Definition,StartOffset,EndOffset
0,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...,PAAF,punción aspiración con aguja fino,1006.0,1010.0
1,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...,RM,resonancia magnético,789.0,791.0
2,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...,cm,centímetro,1345.0,1347.0
3,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...,cm,centímetro,1217.0,1219.0
4,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...,g,gramo,1304.0,1305.0


In [30]:
train = train.rename(columns = {'Abbreviation': 'short_form', 'Definition': 'long_form'})

In [31]:
train = train.dropna(subset = ['short_form', 'long_form'])

Esta vez no eliminamos stopwords ni signos de puntuación del texto en primer lugar, ya que sino no encuentra los SF con los Start y End Offset. Lo haremos después

In [32]:
train[['StartOffset', 'EndOffset']] = train[['StartOffset', 'EndOffset']].astype(int)

In [33]:
train['long_form'] = train['long_form'].map(clean_text)

Transform dataframe to a list of dictionaries

In [36]:
train_data = train[['doc_id', 'texto', 'short_form', 'long_form', 'StartOffset', 'EndOffset']].to_dict('records')

### Cogemos los contextos

In [34]:
def get_f_b_context_text(data):
    
    '''Return context in a string format'''
    
    all_data = []
    sf_not_found = []
    
    for instance in data:
           
        texto = instance['texto']    
        target_word = instance['short_form'] 
        
        if target_word in texto:

            stop_ini_idx = instance['StartOffset'] #índice del inicio de la target
            stop_fin_idx = instance['EndOffset'] #índice del inicio de la target

            _instance = []
            xf = texto[:stop_ini_idx] #palabras anteriores a la target
            xb = texto[stop_fin_idx+1:]   #palabras posteriores a la target            

            instance_id = instance['long_form'] #id del significado

            _instance.append(target_word)
            _instance.append(xf)
            _instance.append(xb)
            _instance.append(instance_id)

            all_data.append(_instance[:])
        else:
            sf_not_found.append(target_word)
#             print("El acrónimo {} no aparece en el texto {}".format(target_word, instance['doc_id']))
        
    return all_data, sf_not_found

In [37]:
train_ndata, sf_not_found = get_f_b_context_text(train_data)

In [36]:
sf_not_found_set = set(sf_not_found)

In [37]:
len(sf_not_found_set)

0

In [38]:
train_ndata[0]

['PAAF',
 'Acude a nuestras consultas a un paciente que presenta una tumoración cervical izquierda de gran tamaño. Como antecedentes destaca intervención quirúrgica de un quiste branquial en esta misma localización hace 18 años.\nExplica lento e indoloro crecimiento de la masa de aproximadamente 10 años de evolución.\nLos límites de la tumoración comprenden cuerpo mayor izquierdo de hioides a nivel anterior, región mastoidea a nivel superior, clavícula en la parte inferior y protuberancia occipital externa como límite posterior.\nLa consistencia de la masa es blanda y no se encuentra adherida a planos profundos.\nNo se palparon adenopatías ni otras masas y la exploración intraoral fue normal.\nEl resto de la exploración física fue completamente anodina.\nSe decide realizar resonancia magnética (RM) cérvico-facial en la que se evidencia una masa homogénea, bien delimitada que no invadía estructuras vecinas y con una densidad grasa.\nSe realiza la toma de una muestra mediante punción-asp

Ahora que ya tenemos los contextos anterior y posterior, limpiamos los textos

In [39]:
def clean_text_after(data):
    
    for instance in data:
        instance[1] = clean_text(instance[1]).lower()
        instance[2] = clean_text(instance[2]).lower()
        instance[1] = re.sub(r'(\d+)', r'\g<1> ', instance[1]) #metemos espacio entre número y acrónimos que quedan pegados    
        instance[2] = re.sub(r'(\d+)', r'\g<1> ', instance[2])
        instance[1] = re.sub(r'\s+', ' ', instance[1]).strip()
        instance[2] = re.sub(r'\s+', ' ', instance[2]).strip()
    return data
        

In [40]:
train_ndata = clean_text_after(train_ndata)

Limitamos las palabras del contexto

In [41]:
n_step_f = 20
n_step_b = 20

def limit_context(data):

    for doc in data:
        
        sf, xf, xb, lf = doc[0], doc[1], doc[2], doc[3]

        xf_words = word_tokenize(xf)[-n_step_f:]
        xb_words = word_tokenize(xb)[:n_step_b]

        doc[1] = ' '.join(xf_words)
        doc[2] = ' '.join(xb_words)   

    return data        

In [42]:
train_ndata[0:2]

[['PAAF',
  'acude consultas paciente presenta tumoracion cervical izquierda gran tamano como antecedentes destaca intervencion quirurgica quiste branquial misma localizacion hace 18 anos explica lento indoloro crecimiento masa aproximadamente 10 anos evolucion los limites tumoracion comprenden cuerpo mayor izquierdo hioides nivel anterior region mastoidea nivel superior clavicula parte inferior protuberancia occipital externa limite posterior la consistencia masa blanda encuentra adherida planos profundos no palparon adenopatias masas exploracion intraoral normal el resto exploracion fisica completamente anodina se decide realizar resonancia magnetica rm cervicofacial evidencia masa homogenea bien delimitada invadia estructuras vecinas densidad grasa se realiza toma muestra mediante puncionaspiracion aguja fina',
  'confirmo sospechas iniciales lipoma se decidio realizar escision quirurgica tumoracion se realizo abordaje nivel linea posterior cabello resecando huso cutaneo 510 cm mejo

In [42]:
train_ndata = limit_context(train_ndata)

### Agrupamos por SF

In [43]:
def create_dict(data):
    
    data_dic = []
    
    for instance in data:
            
        dic = {}

        dic['short_form'] = instance[0]
        dic['context'] = instance[1] + ' ' + instance[2]
        dic['long_form'] = instance[3]
    
        data_dic.append(dic)
        
    return data_dic
    

In [44]:
data_dic = create_dict(train_ndata)

In [45]:
data_df = pd.DataFrame(data_dic)

In [46]:
data_df = data_df.assign(id=(data_df['long_form']).astype('category').cat.codes)

In [47]:
data_df.head()

,short_form,context,long_form,id
0,PAAF,rm cervicofacial evidencia masa homogenea bien...,puncion aspiracion aguja fino,597
1,RM,planos profundos no palparon adenopatias masas...,resonancia magnetico,625
2,cm,posterior cabello resecando huso cutaneo 510 c...,centimetro,125
3,cm,sospechas iniciales lipoma se decidio realizar...,centimetro,125
4,g,se realizo abordaje nivel linea posterior cabe...,gramo,328


In [48]:
group = data_df.groupby('short_form').agg({'long_form': ['nunique', 'size', set]})
group.columns = ['unique', 'size', 'lista']

In [49]:
group[group['unique'] == 2].sort_values('size', ascending=False).head(50)

,unique,size,lista
short_form,,,
cm,2,217,"{centimetro, centimetro agua}"
mm,2,114,"{milimetro, milimetro mercurio}"
h,2,97,"{hora, microgramo}"
kg,2,85,"{kilogramo, centimetro}"
OI,2,79,"{ojo izquiero, ojo izquierdo}"
TC,2,48,"{tomografia computarizado, tomografia computer..."
VSG,2,19,"{velocidad eritrosedimentacion, velocidad sedi..."
PAAF,2,14,"{puncion aspiracion aguja fino, puncion aspira..."
P,2,13,"{phosphorus, fosforo}"


In [50]:
data_df[data_df['short_form'] == 'QT']

,short_form,context,long_form,id
600,QT,dentro valores referencia posteriormente reali...,q timir,599
902,QT,infiltracion lh la tac mostraba adenopatias ax...,quimioterapia,603
903,QT,consolido respuesta 3 ciclos qt regimen mopp r...,quimioterapia,603
904,QT,esclerosis pala iliaca izquierda biopsia medul...,quimioterapia,603
1492,QT,unidad cuidados intensivos nuevo ecg confirma ...,q timir,599
1493,QT,vo dia disminuyendo dosis nitroglicerina 5 ml ...,q timir,599
1494,QT,unidad cuidados intensivos nuevo ecg confirma ...,quimioterapia,603
1495,QT,vo dia disminuyendo dosis nitroglicerina 5 ml ...,quimioterapia,603


In [51]:
prueba = data_df[data_df['short_form'] == 'QT']

In [52]:
prueba

,short_form,context,long_form,id
600,QT,dentro valores referencia posteriormente reali...,q timir,599
902,QT,infiltracion lh la tac mostraba adenopatias ax...,quimioterapia,603
903,QT,consolido respuesta 3 ciclos qt regimen mopp r...,quimioterapia,603
904,QT,esclerosis pala iliaca izquierda biopsia medul...,quimioterapia,603
1492,QT,unidad cuidados intensivos nuevo ecg confirma ...,q timir,599
1493,QT,vo dia disminuyendo dosis nitroglicerina 5 ml ...,q timir,599
1494,QT,unidad cuidados intensivos nuevo ecg confirma ...,quimioterapia,603
1495,QT,vo dia disminuyendo dosis nitroglicerina 5 ml ...,quimioterapia,603


In [83]:
textos = prueba['context'].values
labels = prueba['id'].values

In [57]:
#textos = [np.array([x]) for x in textos]

In [84]:
prueba_data = zip(textos, labels)

In [79]:
# batch_size = 8
# seed = 42

# raw_train_ds = preprocessing.text_dataset_from_directory(
#     prueba,
#     batch_size=batch_size,
#     seed=seed)

### Modelo prueba 2 sentidos

In [55]:
VOCAB_SIZE = 1000 #número máximo de palabras
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(textos)

In [61]:
vocab = np.array(encoder.get_vocabulary())
print(len(vocab))
vocab[:50]

176


array(['', '[UNK]', 'tratamiento', 'mg', 'paciente', 'dia', 'vo',
       'repolarizacion', 'h', 'ecg', 'sinusal', 'regimen', 'nuevo', 'm2',
       'lpm', 'espacio', 'el', 'ciclos', 'abvd', 'valores', 'unidad',
       'unas', 'torno', 'taquicardia', 'tac', 't', 'suspension',
       'sindrome', 'sertralina', 'ser', 'se', 'rt', 'respuesta',
       'regimenes', 'recaida', 'realiza', 'qt', 'prolongacion',
       'progreso', 'presenta', 'posterior', 'persistente', 'ondas',
       'normalizacion', 'nitroglicerina', 'negativas', 'msg', 'mopp',
       'ml', 'manteniendo'], dtype='<U20')

In [79]:
model = tf.keras.Sequential([
    encoder,  #convierte el texto a una secuencia de índices
    tf.keras.layers.Embedding(     #almacena un vector por palabra
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [80]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [67]:
# def batch_generator(X, Y, batch_size=2):
#     indices = np.arange(len(X)) 
#     batch=[]
#     while True:
# #         # it might be a good idea to shuffle your data before each epoch
#         np.random.shuffle(indices) 
#         for i in indices:
#             batch.append(i)
#             if len(batch)==batch_size:
#                 yield X[batch], Y[batch]
#                 batch=[]

In [68]:
# data_gen = batch_generator(textos, labels, batch_size=32)

In [91]:
# for x,y in data_gen:
#     break

#### Transformamos los datos para que los acepte como input

Opción 1

In [91]:
textos = np.reshape(textos, (textos.shape[0], 1))
labels = np.reshape(labels, (labels.shape[0], 1))

In [92]:
prueba_data = zip(textos, labels)

In [83]:
sample_text = ('dentro valores referencia posteriormente realiza tac craneal aprecian anormalidades presenta electrocardiograma dia ingreso ritmo sinusal normal 79 lpm alargamiento espacio pr a dos dias presenta nuevo convulsion tonicoclonica generalizada minuto duracion tras ser valorado servicio neurologia realiza electroencefalograma resulta ser')
predictions = model.predict(np.array([sample_text]))
print(predictions)

[[2117.5457]]


Opción 2

In [95]:
prueba

,short_form,context,long_form
600,QT,dentro valores referencia posteriormente reali...,q timir
902,QT,infiltracion lh la tac mostraba adenopatias ax...,quimioterapia
903,QT,consolido respuesta 3 ciclos qt regimen mopp r...,quimioterapia
904,QT,esclerosis pala iliaca izquierda biopsia medul...,quimioterapia
1492,QT,unidad cuidados intensivos nuevo ecg confirma ...,q timir
1493,QT,vo dia disminuyendo dosis nitroglicerina 5 ml ...,q timir
1494,QT,unidad cuidados intensivos nuevo ecg confirma ...,quimioterapia
1495,QT,vo dia disminuyendo dosis nitroglicerina 5 ml ...,quimioterapia


In [94]:
target = prueba.pop('id')

In [97]:
dataset = tf.data.Dataset.from_tensor_slices((prueba[['context']].values, target.values))

In [98]:
train_dataset = dataset.batch(1)

#### Entrenamos el modelo

In [99]:
history = model.fit(x=train_dataset, epochs=10)

Epoch 1/10
8/8 [==============================] - 4s 20ms/step - loss: -6.8918 - accuracy: 0.0000e+00
Epoch 2/10
8/8 [==============================] - 0s 17ms/step - loss: -14.2565 - accuracy: 0.0000e+00
Epoch 3/10
8/8 [==============================] - 0s 17ms/step - loss: -22.4564 - accuracy: 0.0000e+00
Epoch 4/10
8/8 [==============================] - 0s 17ms/step - loss: -31.9148 - accuracy: 0.0000e+00
Epoch 5/10
8/8 [==============================] - 0s 16ms/step - loss: -42.8505 - accuracy: 0.0000e+00
Epoch 6/10
8/8 [==============================] - 0s 16ms/step - loss: -56.0758 - accuracy: 0.0000e+00
Epoch 7/10
8/8 [==============================] - 0s 17ms/step - loss: -72.2667 - accuracy: 0.0000e+00
Epoch 8/10
8/8 [==============================] - 0s 17ms/step - loss: -93.2420 - accuracy: 0.0000e+00
Epoch 9/10
8/8 [==============================] - 0s 17ms/step - loss: -122.0837 - accuracy: 0.0000e+00
Epoch 10/10
8/8 [==============================] - 0s 16ms/step - loss: -

### Modelo de red sencilla

In [59]:
def create_model(vocab_size, num_labels):
    model = tf.keras.Sequential([
      layers.Embedding(vocab_size, 64, mask_zero=True),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(num_labels)
    ])
    return model

In [63]:
int_model = create_model(vocab_size=VOCAB_SIZE + 1, num_labels=2)

In [74]:
int_model.compile(
    loss=losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

In [77]:
history = int_model.fit(prueba_data,  epochs=5)

AttributeError: 'str' object has no attribute 'shape'